# Compute Gain

In [2]:
import numpy as np
from numba import njit
import pandas as pd
from tqdm import tqdm
pd.options.display.max_colwidth = 100
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from contextlib import contextmanager
@contextmanager
def initiate_plot(dimx=24, dimy=9):
    plt.rcParams['figure.figsize'] = (dimx, dimy)
    global fig; fig = plt.figure()
    yield
    plt.show()

In [4]:
import straxen

Select the Context among:
 - demo
 - xenon1t_analysis
 - nt_daq_test_analysis
 - strax_workshop_dali

In [5]:
st = straxen.contexts.strax_workshop_dali()

In [6]:
st.select_runs(run_mode='LED*')

Checking data availability: 100%|██████████| 5/5 [00:08<00:00,  1.74s/it]


,comments,detector,end,mode,name,number,processor.DEFAULT.drift_velocity_liquid,processor.DEFAULT.electron_lifetime_liquid,processor.DEFAULT.gains,processor.NeuralNet.PosRecNeuralNet.neural_net_file,...,trigger.total_event_duration,trigger.trigger_monitor_data_format_version,trigger.trigger_monitor_data_location,trigger.truncated_events,user,records_available,event_info_available,events_available,peaks_available,raw_records_available
41,"[{'text': 'LED Calibration step 0 #gain_step0', 'date': 2018-02-19 09:52:13+00:00, 'user': 'moli...",tpc,2018-02-19 10:00:16+00:00,LED_3mus_stable,180219_0952,16979,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.157727e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
42,"[{'text': 'LED Calibration step 1 #gain_step1', 'date': 2018-02-19 10:01:41+00:00, 'user': 'moli...",tpc,2018-02-19 10:09:44+00:00,LED_3mus_stable,180219_1001,16980,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.158277e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
43,"[{'text': 'LED Calibration step 2 #gain_step2', 'date': 2018-02-19 10:11:06+00:00, 'user': 'moli...",tpc,2018-02-19 10:19:09+00:00,LED_3mus_stable,180219_1011,16981,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.141546e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
44,"[{'text': 'LED Calibration step 3 #gain_step3', 'date': 2018-02-19 10:21:18+00:00, 'user': 'moli...",tpc,2018-02-19 10:29:22+00:00,LED_3mus_stable,180219_1021,16982,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.160103e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
45,"[{'text': 'LED Calibration step 4 #gain_step4', 'date': 2018-02-19 10:31:02+00:00, 'user': 'moli...",tpc,2018-02-19 10:39:05+00:00,LED_3mus_stable,180219_1030,16983,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.155318e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
46,"[{'text': 'SPE acceptance top bulk #spe_topbulk', 'date': 2018-02-19 10:50:00+00:00, 'user': 'mo...",tpc,2018-02-19 10:58:02+00:00,LED_3mus_stable,180219_1049,16985,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.138026e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
47,"[{'text': 'SPE acceptance top ring #spe_topring', 'date': 2018-02-19 10:59:21+00:00, 'user': 'mo...",tpc,2018-02-19 11:07:24+00:00,LED_3mus_stable,180219_1059,16986,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,2.123550e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
48,"[{'text': '#Afterpulse mode 0', 'date': 2018-02-19 11:10:28+00:00, 'user': 'molinario'}]",tpc,2018-02-19 11:15:31+00:00,LED_7mus_stable,180219_1110,16987,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,5.072760e+09,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True
49,"[{'text': '#Afterpulse mode 1', 'date': 2018-02-19 11:16:48+00:00, 'user': 'molinario'}]",tpc,2018-02-19 11:31:50+00:00,LED_7mus_stable,180219_1116,16988,0.000133,NaN,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",NaN,...,1.511741e+10,2.0,mongodb://gw:27018/trigger_monitor,0.0,molinario,False,False,False,False,True


In [7]:
dsets = st.select_runs(run_mode='LED*')
len(dsets)

9

In [8]:
#st.data_info('event_info')
print(dsets.tags)
dsets = dsets[(dsets.tags == 'gain*')]
dsets.name

41     gain_step0,_sciencerun2_candidate
42     gain_step1,_sciencerun2_candidate
43     gain_step2,_sciencerun2_candidate
44     gain_step3,_sciencerun2_candidate
45     gain_step4,_sciencerun2_candidate
46    spe_topbulk,_sciencerun2_candidate
47    spe_topring,_sciencerun2_candidate
48     Afterpulse,_sciencerun2_candidate
49     Afterpulse,_sciencerun2_candidate
Name: tags, dtype: object


Series([], Name: name, dtype: object)

In [9]:
st.data_info('raw_records')

,Field name,Data type,Comment
0,channel,int16,Channel/PMT number
1,dt,int16,Time resolution in ns
2,time,int64,Start time of the interval (ns since unix epoch)
3,length,int32,Length of the interval in samples
4,area,int32,Integral in ADC x samples
5,pulse_length,int32,Length of pulse to which the record belongs (without zero-padding)
6,record_i,int16,Fragment number in the pulse
7,baseline,float32,Baseline in ADC counts. data = int(baseline) - data_orig
8,reduction_level,uint8,Level of data reduction applied (strax.ReductionLevel enum)
9,data,"('<i2', (110,))",Waveform data in ADC counts above baseline


In [ ]:
#st.data_info('peaks')

It's possible to load the data as a DataFrame or as an array. The loading takes a while.

In [ ]:
noise_id = '180219_0952'
run_id = '180219_1011'

noise = st.get_array(noise_id, 'raw_records', seconds_range=(0, 20))
df = st.get_array(run_id, 'raw_records', seconds_range=(0, 20))
type(df), df.dtype.names

In [ ]:
ch_100 = df[df['channel'] == 100]

In [ ]:
max_length=max(ch_100['length'])
sum_peaks = np.zeros(max_length)
for p in ch_100:
    n = p['length']
    t0 = p['time']        
    sum_peaks[:n] += p['data'][:n] / p['dt']

In [ ]:
print(max(ch_100['length']))
plt.plot(sum_peaks)
plt.ylabel("Sum waveform (PE / ns)")
plt.xlabel("Sample")
plt.xlim([120,200])
#plt.yscale('log')

In [ ]:
channels = np.arange(248)
ch_bot = np.arange(127,248)
#ch_top_in = np.arange(36,127)
ch_top_in = np.arange(36,46)
ch_top_out = np.arange(0,36)

In [ ]:
Area = []
Area_noise = []

left = 130
end_pos = [175, 177, 179, 181, 183, 185]
for ch in tqdm(ch_top_in): #loop on PMT channel
    arr_ch = df[df['channel'] == ch]
    arr_ch_noise = noise[noise['channel'] == ch]

    Area_ch = []
    Area_noise_ch = []
    area = 0
    area_noise = 0
    for right in end_pos:
        area += arr_ch['data'][:,left:right].sum(axis=1)
        area_noise += arr_ch_noise['data'][:,left:right].sum(axis=1)
        print(area)
        
    Area_ch.append(area)
    Area_noise_ch.append(area_noise)
    
    Area_ch[:] = [x/len(end_pos) for x in Area_ch]
    Area_noise_ch[:] = [x/len(end_pos) for x in Area_noise_ch]
    print(Area_ch)
    
    Area.append(Area_ch)
    Area_noise.append(Area_noise_ch)

In [ ]:
print(len(Area[0]))
#print(arr_ch['data'][:,left:175].sum(axis=1))
#print(arr_ch['data'][:,left:185].sum(axis=1))

In [ ]:
s, b = {},{}

with initiate_plot(20, 80):
    for i, val in enumerate(tqdm(ch_top_in)):
        ax = fig.add_subplot(20, 5, i+1)
        plt.plot([], [], ' ', label='PMT '+str(val))
        s[i], bins, patches = plt.hist(Area[i], bins=100, range=[-500,2000] , histtype='step', color='b', label='Signal')
        b[i], bins, patches = plt.hist(Area_noise[i], bins=100, range=[-500,2000] , histtype='step', color='g', label='Noise')
        #plt.hist(df[df['channel'] == val]['data'][:,left:175].sum(axis=1), bins=100, range=[0,2000] , histtype='step', color='r', label='175')
        
        plt.legend(frameon=False)
        ax.set_xlabel("Area (ADCc)")
        ax.set_ylabel("Counts")
        #ax.set_xlim([0,1500])
        #ax.set_ylim([0,1e4])
        ax.set_yscale('log')
        
    plt.tight_layout()

In [ ]:
#Get means and variances
E_t, E_b, V_t, V_b = [],[],[],[]
mids = 0.5*(bins[1:] + bins[:-1])

for i, val in enumerate(tqdm(ch_top_in)):
    mean_t = np.average(mids, weights=s[i])
    mean_b = np.average(mids, weights=b[i])
    E_t.append(mean_t)
    E_b.append(mean_b)
    V_t.append(np.average((mids - mean_t)**2, weights=s[i]))
    V_b.append(np.average((mids - mean_b)**2, weights=b[i]))

In [ ]:
print(E_t)
print(np.sqrt(V_t))
print(E_b)
print(np.sqrt(V_b))

In [ ]:
class HistogramsForGain(strax.Plugin):
    """Make histograms of signal and noise averaged over different integration lenghts"""

    # Name of the data type this plugin provides
    #provides = 'peak_classification'

    # Data types this plugin requires. Note we don't specify
    # what plugins should produce them: maybe the default PeakBasics
    # has been replaced by another AdvancedExpertBlabla plugin?
    depends_on = ('raw_records',)

    # Numpy datatype of the output
    dtype = []

    # Version of the plugin. Increment this if you change the algorithm.
    __version__ = '0.0.1'

    def compute(self, peaks):
        # Your code here.
        # This function will be called several times with
        # 'peaks' a numpy array of the datatype 'peaks'.
        # Each time you'll see a small part of the run.

        Area = []
        Area_noise = []

        left = 130
        end_pos = [175, 177, 179, 181, 183, 185]
        for ch in tqdm(ch_top_in): #loop on PMT channel
            arr_ch = df[df['channel'] == ch]
            arr_ch_noise = noise[noise['channel'] == ch]

            Area_ch = []
            Area_noise_ch = []
            area = 0
            area_noise = 0
            for right in end_pos:
                area += arr_ch['data'][:,left:right].sum(axis=1)
                area_noise += arr_ch_noise['data'][:,left:right].sum(axis=1)

            Area_ch.append(area)
            Area_noise_ch.append(area_noise)

            Area_ch[:] = [x/len(end_pos) for x in Area_ch]
            Area_noise_ch[:] = [x/len(end_pos) for x in Area_noise_ch]

            Area.append(Area_ch)
            Area_noise.append(Area_noise_ch)
        
        # You have to return a numpy array of the dtype you declared above
        # or, which is often easier, a dictionary we can transform into it
        # (dict keys -> field names, values -> field values)
        return dict(type=np.ones(len(peaks)))


In [ ]:
#integrating between samples
left, right = 130, 180
areas = ch_100['data'][:,left:right].sum(axis=1)

# Small correction for baseline, see strax issue #2
areas += ((ch_100['baseline'] % 1) * (right - left)).round().astype(np.int)

plt.hist(areas, bins=100, range=[0,2000] , histtype='step', color='b')

plt.xlabel("Area (ADCc)")
plt.ylabel("Counts")
#plt.xlim([0,1000])
#plt.ylim([1e-1,5e5])
plt.yscale('log')

###  Not sure what I was doing here 

In [ ]:
def plot_peak(p, t0=None, **kwargs):
    n = p['length']
    if t0 is None:
        t0 = p['time']
    plt.plot((p['time'] - t0) + np.arange(n) * p['dt'],
             p['data'][:n] / p['dt'],
             linestyle='steps-mid',
             **kwargs)
    plt.xlabel("Time (ns)")
    plt.ylabel("Sum waveform (PE / ns)")

plot_peak(peaks[5])

In [ ]:
def plot_peaks(main_i, n_before=0, n_after=0, label_threshold=1000):
    for i in main_i + np.arange(-n_before, n_after + 1):
        p = peaks[i]
        label = None
        if p['area'] > label_threshold:
            label = '%.1f PE, %d ns dt' % (p['area'], p['dt'], )
        plot_peak(p,
                  t0=peaks[main_i]['time'],
                  label=label)
    plt.ylim(0, None)
    plt.legend(loc='upper right')
    plt.yscale('symlog')

plot_peaks(1000, n_after=5, n_before=5)